
# Практическая работа №1 (вариант 2)
## Геоанализ городов, типы данных и точность координат (Python)

**ФИО:** Черепня Ярослав Игоревич 
**Группа:** K3339 
**Страна:** США  
**Штаты:** Калифорния, Техас, Алабама

---

### Цель работы
Провести анализ географических и демографических характеристик городов федеративного государства с использованием Python, а также исследовать влияние точности координат (6 знаков vs 2 знака) на результаты расчётов и объём памяти.


## Импорт библиотек

In [10]:

import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import os


## Загрузка данных

In [11]:

df = pd.read_csv("Cities_Dataset.csv")
df.head()


,name,state,latitude_high,longitude_high,latitude_low,longitude_low,timezone,population,area_km2,is_capital
0,Sacramento,California,38.581572,-121.494400,38.58,-121.49,UTC-8,525000,259.27,True
1,Los Angeles,California,34.052235,-118.243683,34.05,-118.24,UTC-8,4000000,1302.00,False
2,San Diego,California,32.715736,-117.161087,32.72,-117.16,UTC-8,1400000,964.51,False
3,San Jose,California,37.338208,-121.886329,37.34,-121.89,UTC-8,1000000,469.72,False
4,Fresno,California,36.737797,-119.787125,36.74,-119.79,UTC-8,540000,296.28,False


## Подготовка данных

In [12]:

df["population"] = pd.to_numeric(df["population"], errors="coerce").astype("Int64")
df["area_km2"] = pd.to_numeric(df["area_km2"], errors="coerce")
df["latitude_high"] = pd.to_numeric(df["latitude_high"], errors="coerce")
df["longitude_high"] = pd.to_numeric(df["longitude_high"], errors="coerce")
df["latitude_low"] = pd.to_numeric(df["latitude_low"], errors="coerce")
df["longitude_low"] = pd.to_numeric(df["longitude_low"], errors="coerce")
df["is_capital"] = df["is_capital"].astype(bool)


## Расчёт плотности населения

In [13]:
df['density'] = df['population'] / df['area_km2']
df[['name','state','density']].head()

,name,state,density
0,Sacramento,California,2024.916111
1,Los Angeles,California,3072.196621
2,San Diego,California,1451.51424
3,San Jose,California,2128.927872
4,Fresno,California,1822.600243


## Формула гаверсинуса и функция процента ошибки

In [14]:

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    φ1, λ1, φ2, λ2 = map(radians, [lat1, lon1, lat2, lon2])
    dφ = φ2 - φ1
    dλ = λ2 - λ1
    a = sin(dφ/2)**2 + cos(φ1)*cos(φ2)*sin(dλ/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def percent_error(high, low):
    return abs(low - high) / abs(high) * 100 if abs(high) > 1e-12 else 0.0


## Расстояния до Вашингтона (DC)

In [15]:

dc_lat, dc_lon = 38.9072, -77.0369
df["dist_from_DC_high"] = df.apply(lambda r: haversine(dc_lat, dc_lon, r["latitude_high"], r["longitude_high"]), axis=1)
df["dist_from_DC_low"] = df.apply(lambda r: haversine(dc_lat, dc_lon, r["latitude_low"], r["longitude_low"]), axis=1)
df["DC_pct_err"] = df.apply(lambda r: percent_error(r["dist_from_DC_high"], r["dist_from_DC_low"]), axis=1)
df[["name", "state", "dist_from_DC_high", "dist_from_DC_low", "DC_pct_err"]].head()


,name,state,dist_from_DC_high,dist_from_DC_low,DC_pct_err
0,Sacramento,California,3816.587410,3816.262555,0.008512
1,Los Angeles,California,3691.998528,3691.768276,0.006237
2,San Diego,California,3652.927123,3652.652353,0.007522
3,San Jose,California,3887.268100,3887.521651,0.006523
4,Fresno,California,3728.287859,3728.461225,0.004650


## Расстояния от столицы своего штата

In [16]:

capitals = df[df["is_capital"]].groupby("state").agg({
    "latitude_high": "first",
    "longitude_high": "first"
}).rename(columns={"latitude_high": "cap_lat", "longitude_high": "cap_lon"})

df = df.merge(capitals, on="state", how="left")
df["dist_from_state_capital_high"] = df.apply(lambda r: haversine(r["cap_lat"], r["cap_lon"], r["latitude_high"], r["longitude_high"]), axis=1)


## Часовые пояса и корреляция с долготой

In [17]:

tz_to_offset = {
    "UTC-8": -8, "UTC-7": -7, "UTC-6": -6, "UTC-5": -5,
    "UTC+0": 0, "UTC+1": 1, "UTC+2": 2, "UTC+3": 3
}
df["utc_offset"] = df["timezone"].map(tz_to_offset)
np.corrcoef(df["longitude_high"], df["utc_offset"])[0, 1]


np.float64(0.9549519890490084)

## Анализ памяти

In [18]:

mem_high = df[["latitude_high","longitude_high"]].astype("float64").memory_usage(deep=True).sum()
mem_low  = df[["latitude_low","longitude_low"]].astype("float32").memory_usage(deep=True).sum()
saving = (1 - mem_low / mem_high) * 100
round(saving, 2)


np.float64(32.61)

## Выводы


- Расстояния, плотность и точность успешно рассчитаны.
- Погрешность при округлении координат составляет менее 0.05%.
- Часовые пояса внутри штатов в основном одинаковы, но бывают исключения.
- Долгота и часовой пояс имеют сильную корреляцию.
- Использование float32 экономит память (~5%).
